In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
# Add parent dir to path, so that python finds the lenskit package
sys.path.insert(0,parentdir) 

In [2]:
from lenskit.metrics import dataGenerator
from lenskit import batch, topn, util, topnFair
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, funksvd, user_knn as uknn, item_knn as iknn 
from lenskit import topn, topnFair

In [3]:
import numpy as np
import pandas as pd
import math
import pickle
%matplotlib inline

In [4]:
test_data = pd.read_pickle("test_data_Lenskit100k")
all_recs_joined = pd.read_pickle("all_recs_joined_Lenskit100k")

In [5]:
def evaluate_genre(genre): 
    
    providers = list(all_recs_joined.iloc[:,8:])
    proItems_genre_n =  all_recs_joined.loc[ all_recs_joined[genre] == 1].item.nunique()
    print (genre)
    rla = topnFair.FairRecListAnalysis(['user', "Algorithm"])
    rla.add_metric("rND")
    rla.add_metric("rKL")
    rla.add_metric("rRD")
    rla.add_metric("APCR")
    rla.add_metric("nd_APCR")
    rla.add_metric("equal_ex")
    rla.add_metric("ndcg")
    results= rla.compute(all_recs_joined, test_data, genre, providers)
    #results_horror.groupby('Algorithm').mean()
    print("done" , genre)
    return results.groupby('Algorithm').mean()

In [6]:
testproviders = ["Drama", "Action"]
#testproviders = list(all_recs_joined.iloc[:,8:])
allResults = []
for genre in testproviders:
    rep = evaluate_genre(genre)
    allResults.append(rep)    

Drama


/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1472: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


done Drama
Action
done Action


In [7]:
output = pd.concat(allResults, keys=testproviders)
output

rND       rKL       rRD  APCR   nd_APCR  equal_ex  \
       Algorithm                                                           
Drama  ALS        1.060204  1.006622  0.649797   1.0  0.963533  1.006936   
       FunkSVD    1.057966  1.004535  0.508939   1.0  0.962361  1.006587   
       ItemItem   1.055437  1.005861  0.664490   1.0  0.962878  1.000358   
       UserUser   1.066052  1.007750  0.318948   1.0  0.960140  0.983756   
Action ALS        1.242901  1.042514  0.966425   1.0  0.963533  1.019947   
       FunkSVD    1.291129  1.062032  0.949500   1.0  0.962361  1.014297   
       ItemItem   1.226653  1.039724  0.929619   1.0  0.962878  1.004476   
       UserUser   1.244386  1.033283  0.942231   1.0  0.960140  1.009035   

                      ndcg  
       Algorithm            
Drama  ALS        0.320517  
       FunkSVD    0.282647  
       ItemItem   0.296505  
       UserUser   0.271042  
Action ALS        0.320517  
       FunkSVD    0.282647  
       ItemItem   0.296505  
       UserUser   0.271042